In [141]:
#  데이터 처리 모듈
import pandas as pd
from sklearn.impute import KNNImputer
# 행렬 등 통계 연산 모듈
import numpy as np
from scipy.stats import iqr
from sklearn.impute import KNNImputer
# 지수형 표기법 e를 연속형 변환
pd.options.display.float_format = '{:.4f}'.format
# 타입 어노테이션(Any, Sequence 등의 메서드 활용)
from typing import *
import os

# 구글 드라이브 마운트
from google.colab import drive
drive.mount("/content/drive")
import os

# 조합 찾는 반복문 메서드
from itertools import permutations
from itertools import combinations

# 데이터 시각화
import matplotlib.pyplot as plt
# from matplotlib.pyplot import
import matplotlib.font_manager as fm
import missingno as msno
import seaborn as sns

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [142]:
# apt-get : 패키지 설치 명령어
# -qq : 에러외의 메세지 숨기는 옵션키
!apt-get -qq install fonts-nanum

fe = fm.FontEntry(
    fname=r"/usr/share/fonts/truetype/nanum/NanumGothic.ttf",
    # ttf 파일이 저장되어 있는 경로
    name='NanumGothic')
# 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)
# Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 18, 'font.family': 'NanumGothic'})
# 폰트 설정

In [143]:
class LoadGoogleDriveData():
  def __init__(self, data = None):
    self.data = data

  def loadData(self, file_path: str, file_name_extension,
               columnTF: bool, unicode: str) -> pd.DataFrame():
    self.data = pd.read_csv(os.path.join(file_path + file_name_extension),
                            index_col = columnTF,
                            na_values = "NaN",
                            encoding = unicode)
    return self.data

  # 용량이 큰 csv 파일 읽어오기(fopen - fread와 유사한 방식)
  def loadDataWithChunking(self, file_path: str, file_name_extension,
                           chunking_row_num: int, columnTF: bool, unicode: str) -> pd.DataFrame():
    chunkdata = pd.read_csv(os.path.join(file_path + file_name_extension),
                            chunksize = chunking_row_num,
                            index_col = columnTF,
                            na_values = "NaN",
                            encoding = unicode)
    self.data = list(chunkdata)
    self.data = pd.concat(self.data)

    return self.data

In [144]:
mountInstance = LoadGoogleDriveData()

### 1.오아시스 비즈니스(필지고유번호 & 법정동읍면동코드 & 외식업종)  데이터

In [145]:
CONTEST_2023 = mountInstance.loadDataWithChunking(
    file_path = "/content/drive/MyDrive/2023BigContest/data/contest data/", file_name_extension = "소상공인 매출등급 예상 대상 필지.csv",
    unicode = "cp949", chunking_row_num = 10**5, columnTF = False)

In [146]:
CONTEST_2022 = mountInstance.loadDataWithChunking(
    file_path = "/content/drive/MyDrive/2023BigContest/data/contest data/", file_name_extension = "필지단위 소상공인 매출등급 정보.csv",
    unicode = "cp949", chunking_row_num = 10**5, columnTF = False)

### 2. KCD(상권코드 & 외식업종 중분류) 소상공인 신용데이터

In [147]:
KCD_2022_2023 = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/", file_name_extension = "SOHO_MERGE_DATA_LIST.csv",
    unicode = "utf-8", columnTF = False)

### 3. 행정동코드 - 상권코드 매핑 데이터
```
KCD_MERGE_LIST[0] = KCD_MERGE_LIST[0].drop(["상권_코드_2022"], axis = 1)
KCD_MERGE_LIST[0].drop_duplicates(subset = ["STDG_EMD_CD"], inplace = True)
KCD_MERGE_LIST[0].isnull().sum()
  # 상권코드           431
  # 매출액_변동계수       431
KCD_MERGE_LIST[0].isnull().sum()
  # 상권코드               404
len(MARKET_LOC_MAP["상권_코드_2022"].unique())
  # 1622
len(MARKET_LOC_MAP['STDG_EMD_CD'].unique())
  # 465
```

In [148]:
MARKET_LOC_MAP = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/",
    file_name_extension = "market_loc_map.csv",
    unicode = "euc-kr", columnTF = False)

In [149]:
MARKET_LOC_MAP

,상권_코드_2022,상권_코드_2023,행정동_코드,법정동_코드,STDG_EMD_CD
0,2110008,3110008,11110515,11110101,11110101
1,2110008,3110008,11110515,11110102,11110102
2,2110008,3110008,11110515,11110103,11110103
3,2110008,3110008,11110515,11110104,11110104
4,2110008,3110008,11110515,11110105,11110105
...,...,...,...,...,...
3453,2130102,3130102,11230750,11230110,11230110
3454,2130166,3130166,11380590,11380107,11380107
3455,2130113,3130113,11260690,11260106,11260106
3456,2130124,3130124,11290780,11290138,11290138


### 4. 서울열린데이터광장 상권특성 공공데이터

### 5.분기별 상권특성 공공데이터 전처리

### 6. 분기별 상권별 상권특성 공공데이터 병합

### 7. 소호 신용데이터(KCD) 이상치 처리 및 결측값 대체

* KCD_NUM_LIST : 연속형 변수만 추출하여 이상치를 결측값으로 처리한 데이터의 리스트(IQR)
* KCD_IMPUTED_DATA : 유사도가 높은 컬럼별 결측값을 최근접 이웃 5개 데이터의 평균값으로 대체한 데이터의 리스트(KNN, K = 5)
* KCD_PREPROCESS_DATA : 전처리 완료된 데이터에 상권코드, 상권명, 기준일자 등 인덱스 포함한 데이터의 리스트

In [150]:
KCD_DATA = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/", file_name_extension = "KCD_DATA.csv",
    unicode = "euc-kr", columnTF = False)

In [151]:
KCD_DATA

,기준일자,상권_식별자,상권코드,배달매출액_변동계수,주말배달매출액_변동계수,손익분기점매출액_변동계수,사업장방문고객수_평균,정규고용인원_평균,주말카드매출액_변동계수,매입액_변동계수,사업장임대면적_변동계수,부가가치세_변동계수,객단가_변동계수,신규고객단가_변동계수,매출총이익_변동계수,부가세차감전영업이익_변동계수,소상공인업력_평균,프랜차이즈사업장_총합,소상공인연령_평균,소호사업장_수,경영위기사업장_총합,사업장방문신규고객수_평균,월임대료_변동계수,임대보증금_변동계수,매출액_변동계수,카드매출액_변동계수,사업장방문고객수_변동계수,사업장방문신규고객수_변동계수,상권명,외식업종_중분류
0,202201,DMC(디지털미디어시티)_카페,2120098,2.1558,2.5660,0.5387,6.3708,0.8292,1.3211,0.7483,0.4128,-0.6996,0.7058,0.9996,0.9649,1.1789,5.2692,3.0000,39.5846,13.0000,0.0000,6.6692,0.5341,0.5648,0.4986,0.4845,0.7065,0.4988,DMC(디지털미디어시티),카페
1,202201,가락시장역_한식,2120234,1.8341,1.9303,0.4745,4.9360,2.3500,1.4125,0.5770,0.5250,-0.7722,0.9883,1.0187,1.0797,1.1607,5.8400,3.0000,44.9400,5.0000,0.0000,3.3360,0.5646,0.5735,1.0273,1.1870,1.0395,1.0085,가락시장역,한식
2,202201,가로수길_술집,2120186,2.4496,2.4495,0.7954,3.3886,1.0431,1.6146,0.8874,0.5326,-1.3127,0.8118,0.8000,1.5937,1.8294,5.9000,1.0000,42.3571,7.0000,0.0000,2.9971,1.0291,1.1673,0.8737,0.9827,1.1755,0.9964,가로수길,술집
3,202201,가로수길_일식,2120186,2.4495,2.4495,0.3542,6.6600,1.0431,0.3888,0.6522,0.6296,-0.6643,0.5502,0.5487,0.4792,0.4953,3.5714,1.0000,34.6571,7.0000,0.0000,5.9857,0.3630,0.4541,0.4905,0.4996,0.8916,0.8940,가로수길,일식
4,202201,가로수길_한식,2120186,2.0289,2.0816,0.2583,9.8900,1.0431,1.0227,0.6220,0.3451,-0.5937,0.7582,0.7741,0.6349,0.6890,4.6429,2.0000,41.9143,7.0000,0.0000,6.4200,0.2461,0.3202,0.6021,0.6021,0.7941,0.7777,가로수길,한식
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2687,202303,홍대입구역(홍대)_한식,3120103,2.0752,1.8920,0.7577,7.5168,1.6514,1.0790,1.0970,0.8892,-0.9489,1.2670,1.2298,1.0959,1.1860,3.9571,2.0000,37.7571,11.2000,3.0000,5.7888,0.6365,0.7507,0.9257,1.0410,0.7307,0.7528,홍대입구역(홍대),한식
2688,202303,선릉역_카페,3120210,2.1606,2.2517,0.5606,7.7299,1.2906,0.9155,0.7616,0.5936,-0.7855,0.9229,0.9638,0.8262,0.9013,4.2167,1.0000,35.3167,6.0000,1.0000,5.3143,0.5803,0.6383,0.7098,0.7216,0.7822,0.7560,선릉역,카페
2689,202303,"신촌역(신촌역, 신촌로터리)_카페",3120094,2.1606,2.2517,0.5606,7.7299,1.2906,0.9155,0.7616,0.5936,-0.7855,0.9229,0.9638,0.8262,0.9013,5.2400,1.0000,42.8600,5.0000,0.0000,5.3143,0.5803,0.6383,0.7098,0.7216,0.7822,0.7560,"신촌역(신촌역, 신촌로터리)",카페
2690,202303,"총신대입구역(이수, 총신대)_카페",3120156,2.1606,2.2517,0.5606,7.7299,1.2906,0.9155,0.7616,0.5936,-0.7855,0.9229,0.9638,0.8262,0.9013,3.9800,2.0000,32.8800,5.0000,2.0000,5.3143,0.5803,0.6383,0.7098,0.7216,0.7822,0.7560,"총신대입구역(이수, 총신대)",카페


### 8. 분기별 상권데이터를 소호 신용데이터를 활용하여 월별로 확장(계절성 지수)

In [152]:
MARKET_2022_GROUPED = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/", file_name_extension = "MARKET_2022_JOIN_GROUPED.csv",
    unicode = "utf-8", columnTF = False)

In [153]:
# MARKET_2022_GROUPED.drop("2022년도_상권_코드", axis = 1, inplace = True)
MARKET_2022_GROUPED[['기준_년_코드', '기준_분기_코드']] = MARKET_2022_GROUPED[['기준_년_코드', '기준_분기_코드']].astype("str")
MARKET_2022_GROUPED["기준일자"] = MARKET_2022_GROUPED['기준_년_코드'] + MARKET_2022_GROUPED['기준_분기_코드'] + "Q"

In [154]:
MARKET_2023_GROUPED = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/", file_name_extension = "MARKET_2023_JOIN_GROUPED.csv",
    unicode = "utf-8", columnTF = False)

In [155]:
MARKET_2023_GROUPED[['기준_년_코드', '기준_분기_코드']] = MARKET_2023_GROUPED[['기준_년_코드', '기준_분기_코드']].astype("str")
MARKET_2023_GROUPED["기준일자"] = MARKET_2023_GROUPED['기준_년_코드'] + MARKET_2023_GROUPED['기준_분기_코드'] + "Q"

In [174]:
class QuartertoMonth():
  def __init__(self, KCD_dataframe, MARKET_2022, MARKET_2023, KCD_COLUMNS = None):
    self.KCD_dataframe = KCD_dataframe
    self.KCD_COLUMNS = KCD_COLUMNS
    self.MARKET_2022 = MARKET_2022
    self.MARKET_2023 = MARKET_2023

    self.dataframe_month_list = []
    self.dataframe_quarter_list = []
    self.dataframe_market_list = []

  def splitKCDQuarterData(self, base_date: str, date_time1, date_time2,
                       remove_columns: List, groupby_standard_columns: List):
    # 월별 소호 신용데이터를 분기별로 만들기 위한 집계 연산(GROUP BY)을 수행할 컬럼
    self.KCD_COLUMNS = self.KCD_dataframe.columns.tolist()
    for column in remove_columns:
      self.KCD_COLUMNS.remove(column)

    # 월별 시계열을 원소로 담는 리스트 컴프리핸션
    date_time_list = [str(date_time1 + n) for n in range(0, 12, 1)]
    date_time_list.extend([str(date_time2 + n) for n in range(0, 3, 1)])
    date_time_list = [int(date_time) for date_time in date_time_list]

    # 데이터를 월별로 분할 후 분기별로 병합하여 멤버변수로 저장
    for i in range(len(self.KCD_dataframe)):
      self.dataframe_month_list.append(
          self.KCD_dataframe[ self.KCD_dataframe[base_date].isin(date_time_list[i:i+3]) ])
    for index, quarter in zip( list(range(0, 5, 1)), ["20221Q", "20222Q", "20223Q", "20224Q", "20231Q"]):
      index_data = self.dataframe_month_list[index].groupby(groupby_standard_columns)[self.KCD_COLUMNS].sum()
      index_data.reset_index(inplace = True)
      index_data["기준분기"] = quarter
      self.dataframe_quarter_list.append(index_data)
      index_data = 0
    return self.dataframe_quarter_list

  def splitMARKETuarterData(self, base_date: str):
    for index in self.MARKET_2022[base_date].unique():
      self.dataframe_market_list.append(self.MARKET_2022[self.MARKET_2022[base_date] == index])
    self.dataframe_market_list.append(self.MARKET_2023)
    return self.dataframe_market_list

  def seasonalPattern(self, base_date: str):
    SEASONAL_PATTERN = self.KCD_dataframe.groupby([base_date])[self.KCD_COLUMNS].mean()
    SEASONAL_PATTERN = SEASONAL_PATTERN / SEASONAL_PATTERN.mean()
    SEASONAL_PATTERN.reset_index(inplace = True)
    return SEASONAL_PATTERN

In [175]:
mappingTimeseries = QuartertoMonth(KCD_dataframe = KCD_DATA,
                                   MARKET_2022 = MARKET_2022_GROUPED,
                                   MARKET_2023 = MARKET_2023_GROUPED)

In [176]:
KCD_LIST = mappingTimeseries.splitKCDQuarterData(
     base_date = "기준일자",
     date_time1 = 202201, date_time2 = 202301,
     remove_columns = ['기준일자','상권_식별자','상권코드','상권명','외식업종_중분류','소상공인연령_평균','소상공인업력_평균'],
     groupby_standard_columns = ["상권_식별자", "상권코드", "상권명", "외식업종_중분류"])

In [177]:
MARKET_LIST = mappingTimeseries.splitMARKETuarterData(base_date = "기준일자")

In [178]:
SEASON_PATTERN = mappingTimeseries.seasonalPattern(base_date = "기준일자")

In [179]:
pd.set_option("display.max_column", None)

In [180]:
# KCD_LIST[4], MARKET_LIST[0], SEASON_PATTERN

In [181]:
# KCD_LIST[0]["상권코드"], MARKET_LIST[0]["상권_코드"]
# msno.bar(MERGE_LIST[0])

---

In [185]:
CODE = pd.merge(CONTEST_2022[["STDG_EMD_CD"]], MARKET_LOC_MAP, how = "inner")
# CODE = pd.merge(CONTEST_2023[["STDG_EMD_CD"]], MARKET_LOC_MAP, how = "inner")
# KCD_LIST[4]["상권코드"] = KCD_LIST[4]["상권코드"] + 1000000
KCD_LIST[4]
# KCD_MERGE_LIST
KCD_MERGE_LIST = []

for index in range(len(KCD_LIST)):
  KCD_MERGE = pd.merge(left = KCD_LIST[index][["상권코드", "매출액_변동계수"]], right = CODE[["상권_코드_2022", "STDG_EMD_CD"]],
                       left_on = "상권코드", right_on = "상권_코드_2022", how = "outer")
  KCD_MERGE_LIST.append(KCD_MERGE)
  KCD_MERGE = 0

for index in range(len(KCD_MERGE_LIST)):
  KCD_MERGE_LIST[index] = KCD_MERGE_LIST[index].drop(["상권_코드_2022", "매출액_변동계수"], axis = 1)
  KCD_MERGE_LIST[index].drop_duplicates(subset = ["STDG_EMD_CD"], inplace = True)
  KCD_MERGE_LIST[index].reset_index(drop = True, inplace = True)
  print(KCD_MERGE_LIST[index].isnull().sum())
  # KCD_MERGE_LIST[0][KCD_MERGE_LIST[0]["상권코드"].isna()]

display(KCD_MERGE_LIST[4])

KCD_JOIN_LIST = []

for index in range(0, 5, 1):
  join_data = pd.merge(left = KCD_MERGE_LIST[index], right = KCD_LIST[index], on = "상권코드", how = "left")
  join_data.drop_duplicates(subset = ["STDG_EMD_CD"], inplace = True)
  join_data.reset_index(drop = True, inplace = True)
  KCD_JOIN_LIST.append(join_data)
  print(len(KCD_JOIN_LIST[index]))
  # print(KCD_JOIN_LIST[index].isnull().sum())

MERGE_LIST = []

for index in range(0, 5, 1):
  merge_data = pd.merge(
      left = KCD_JOIN_LIST[index], right = MARKET_LIST[index],
      left_on = "상권코드", right_on = "상권_코드",
      how = "left")
  MERGE_LIST.append(merge_data)

MERGE_LIST[0]
# pd.concat(MERGE_LIST).to_csv("/content/drive/MyDrive/2023BigContest/KCD_MARKET_CONTEST2022.csv", index = False)

상권코드           274
STDG_EMD_CD      1
dtype: int64
상권코드           274
STDG_EMD_CD      1
dtype: int64
상권코드           274
STDG_EMD_CD      1
dtype: int64
상권코드           274
STDG_EMD_CD      1
dtype: int64
상권코드           460
STDG_EMD_CD      1
dtype: int64


,상권코드,STDG_EMD_CD
0,3120098.0000,NaN
1,NaN,11110101.0000
2,NaN,11110102.0000
3,NaN,11110103.0000
4,NaN,11110104.0000
...,...,...
456,NaN,11740105.0000
457,NaN,11740106.0000
458,NaN,11740107.0000
459,NaN,11740108.0000


461
461
461
461
461


,상권코드,STDG_EMD_CD,상권_식별자,상권명,외식업종_중분류,배달매출액_변동계수,주말배달매출액_변동계수,손익분기점매출액_변동계수,사업장방문고객수_평균,정규고용인원_평균,주말카드매출액_변동계수,매입액_변동계수,사업장임대면적_변동계수,부가가치세_변동계수,객단가_변동계수,신규고객단가_변동계수,매출총이익_변동계수,부가세차감전영업이익_변동계수,프랜차이즈사업장_총합,소호사업장_수,경영위기사업장_총합,사업장방문신규고객수_평균,월임대료_변동계수,임대보증금_변동계수,매출액_변동계수,카드매출액_변동계수,사업장방문고객수_변동계수,사업장방문신규고객수_변동계수,기준분기,기준_년_코드,기준_분기_코드,상권_코드,상권_코드_명,월_평균_소득_금액,소득_구간_코드,지출_총금액,식료품_지출_총금액,의류_신발_지출_총금액,생활용품_지출_총금액,의료비_지출_총금액,교통_지출_총금액,여가_지출_총금액,문화_지출_총금액,교육_지출_총금액,유흥_지출_총금액,아파트_단지_수,아파트_가격_1_억_미만_세대_수,아파트_가격_1_억_세대_수,아파트_가격_2_억_세대_수,아파트_가격_3_억_세대_수,아파트_가격_4_억_세대_수,아파트_가격_5_억_세대_수,아파트_가격_6_억_이상_세대_수,점포_수,유사_업종_점포_수,개업_율,개업_점포_수,폐업_률,폐업_점포_수,프랜차이즈_점포_수,집객시설_수,관공서_수,은행_수,종합병원_수,일반_병원_수,약국_수,유치원_수,초등학교_수,중학교_수,고등학교_수,대학교_수,백화점_수,슈퍼마켓_수,극장_수,숙박_시설_수,공항_수,철도_역_수,버스_터미널_수,지하철_역_수,버스_정거장_수,총_생활인구_수,남성_생활인구_수,여성_생활인구_수,연령대_10_생활인구_수,연령대_20_생활인구_수,연령대_30_생활인구_수,연령대_40_생활인구_수,연령대_50_생활인구_수,연령대_60_이상_생활인구_수,시간대_1_생활인구_수,시간대_2_생활인구_수,시간대_3_생활인구_수,시간대_4_생활인구_수,시간대_5_생활인구_수,시간대_6_생활인구_수,월요일_생활인구_수,화요일_생활인구_수,수요일_생활인구_수,목요일_생활인구_수,금요일_생활인구_수,토요일_생활인구_수,일요일_생활인구_수,상권_코드_2022,상권_코드_2023,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균,기준일자
0,2120098.0000,11440127.0000,DMC(디지털미디어시티)_카페,DMC(디지털미디어시티),카페,6.6481,7.9662,1.6407,17.8384,2.4877,3.9457,2.6117,1.2383,-1.9385,2.0967,2.8979,2.5806,3.1428,9.0000,39.0000,5.0000,18.4962,1.6023,1.6943,1.5432,1.5156,2.2109,1.6058,20221Q,2022,1,2120098.0000,DMC(디지털미디어시티),3567709.0000,7.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,9.3231,11.7846,1.2154,0.2923,1.4923,0.1538,2.4615,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,318202.0000,166599.0000,151603.0000,28268.0000,68698.0000,84321.0000,72802.0000,37946.0000,26166.0000,29352.0000,68100.0000,66954.0000,69368.0000,61400.0000,23033.0000,53772.0000,52025.0000,51969.0000,56139.0000,52455.0000,27161.0000,24683.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,20221Q
1,2120234.0000,11710107.0000,가락시장역_한식,가락시장역,한식,5.8697,6.1537,1.2814,14.1810,6.4333,4.2607,1.5821,1.4835,-2.6248,2.9556,3.2020,3.2779,3.5323,9.0000,17.0000,4.0000,9.7910,1.7031,1.6620,3.0895,3.5132,3.1365,2.8977,20221Q,2022,1,2120234.0000,가락시장역,3494644.0000,7.0000,1665103832.0000,390216327.0000,206562616.0000,128190190.0000,195855624.0000,292464031.0000,70879876.0000,47683885.0000,262526207.0000,70725076.0000,20.0000,41.0000,194.0000,50.0000,7.0000,48.0000,39.0000,294.0000,12.9176,14.1529,2.1412,0.2118,0.9412,0.2941,1.2353,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1430185.0000,731545.0000,698640.0000,139095.0000,195641.0000,300251.0000,271714.0000,254540.0000,268946.0000,299144.0000,291914.0000,209492.0000,208104.0000,259763.0000,161771.0000,207551.0000,207070.0000,208530.0000,214831.0000,215912.0000,195928.0000,180363.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,20221Q
2,2120186.0000,11680107.0000,가로수길_술집,가로수길,술집,7.3501,7.3485,2.5290,10.4700,3.1739,4.6111,2.5521,1.5978,-3.5006,2.3615,2.3486,4.5132,5.1417,3.0000,21.0000,3.0000,9.3329,3.0872,3.5018,3.4754,3.5867,3.2850,3.0765,20221Q,2022,1,2120186.0000,가로수길,3602514.0000,7.0000,2489687551.0000,589883506.0000,309226730.0000,199643898.0000,284332831.0000,434636261.0000,102125919.0000,76064185.0000,386594435.0000,107179786.0000,62.0000,0.0000,154.0000,140.0000,109.0000,74.0000,49.0000,97.0000,18.4471,19.6118,1.9882,0.4353,1.9412,0.2471,1.1647,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1977135.0000,911097.0000,1066039.0000,212675.0000,462621.0000,456964.0000,352892.0000,224746.0000,267239.0000,274152.0000,342832.0000,341513.0000,384316.0000,455411.0000,178912.0000,284373.0000,284850.0000,288722.0000,300683.0000,308841.0000,274353.0000,235313.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,20221Q
3,

In [186]:
KCD_20221Q_20224Q_JOINDATA = pd.concat(MERGE_LIST[0:4], axis = 0)
KCD_20231Q_JOINDATA = MERGE_LIST[4]

In [192]:
pd.concat([KCD_20221Q_20224Q_JOINDATA, KCD_20231Q_JOINDATA], axis = 0).to_csv("/content/drive/MyDrive/2023BigContest/KCD_MARKET_CONTEST_JOIN.csv", index = False)